In [3]:
from pyarc.data_structures import ClassAssocationRule
from pyids.data_structures import (
    IDSRule,
    IDSRuleSet,
    IDSClassifier,
    IDSObjectiveFunction,
    ObjectiveFunctionParameters,
    ObjectiveFunctionParameters,
    RSOptimizer,
    DLSOptimizer,
    SLSOptimizer,
    RandomizedUSMOptimizer,
    DeterministicUSMOptimizer
)

In [4]:
def print_cars(ids_cars):
    cars = ids_cars
    is_ids = False
    if type(ids_cars[0]) == IDSRule:
        is_ids = [ r.calc_f1() for r in ids_cars ]
        cars = [ r.car for r in ids_cars ]
    latex = []
    for idx, car in enumerate(cars):
        if is_ids:
            latex.append(car_to_latex(car, is_ids=is_ids[idx]))
        else:
            latex.append(car_to_latex(car))
        
    return "\n\n".join(latex)
        
def car_to_latex(car, is_ids=False):
    trans = {
        "-":  r"\-",
        "_":  r"\_",
        "]":  r"\]",
        "\\": r"\\",
        "^":  r"\^",
        "$":  r"\$",
        "*":  r"\*",
        ".":  r"\."
    }
    latex = []
    
    latex.append("If")
    
    for idx, predicate in enumerate(car.antecedent):
       
        name, interval = predicate
        name = name.translate(str.maketrans(trans))
        
        interval_string = interval.translate(str.maketrans(trans))
        
        if idx != 0:
            latex.append("and")
        latex.append(
            f"\\textcolor{{blue}}{{{name}}}=\\textcolor{{gray}}{{{interval_string}}}"
        )

        
    class_name, class_value = car.consequent
    
    latex.append("then")
    latex.append(
        f"\\textcolor{{red}}{{{class_name}}}=\\textcolor{{gray}}{{{class_value}}}"
    )
    
    if is_ids:
        latex.append(
            f"$\mid$ f1-score: \\textit{{{round(is_ids, 2)}}}"
        )
    
    return " ".join(latex)

# IDS Demonstration

## Importing required libraries

In [5]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.data_structures import mine_CARs
from pyids.rule_mining import RuleMiner
from pyarc.qcba.data_structures import QuantitativeDataFrame

from sklearn.model_selection import train_test_split

## Loading the test and train data

In [6]:
data = pd.read_csv("../data/titanic.csv").sample(frac=1).reset_index(drop=True)


data_train, data_test = train_test_split(data, test_size=0.8)

In [7]:
data.head()

,Passenger_Cat,Age_Cat,Gender,Died
0,3rd_class,adult,male,1
1,1st_class,adult,male,0
2,3rd_class,adult,male,1
3,3rd_class,child,male,1
4,3rd_class,adult,male,1


In [8]:
cars = mine_CARs(data_train, 20)
car = cars[0]

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=4
Rule count: 50, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 2
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 3
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 4
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 5
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000000000004, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 6
Decreasing confidence to 0.25000000000000006
Running apriori

In [9]:
ids_rule = IDSRule(car)

In [10]:
quant_dataframe_train = QuantitativeDataFrame(data_train)
quant_dataframe_test = QuantitativeDataFrame(data_test)

D:\programy\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
type(quant_dataframe_train)

pyarc.qcba.data_structures.quant_dataset.QuantitativeDataFrame

In [12]:
ids_rule.cache_prepared

False

In [13]:
ids_rule.cover_cache

{'cover': None,
 'correct_cover': None,
 'incorrect_cover': None,
 'rule_cover': None}

In [14]:
ids_rule1 = IDSRule(cars[0])
ids_rule1.cover_cache

{'cover': None,
 'correct_cover': None,
 'incorrect_cover': None,
 'rule_cover': None}

In [15]:
ids_rule1 = IDSRule(cars[0])
ids_rule2 = IDSRule(cars[1])

print(ids_rule1)
print(ids_rule2)

IDSRule {Gender=female,Passenger_Cat=1st_class} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 3, id: 41
IDSRule {Passenger_Cat=1st_class,Gender=female,Age_Cat=adult} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 4, id: 40


In [16]:
ids_rule1.calculate_cover(quant_dataframe_train)
ids_rule2.calculate_cover(quant_dataframe_train)

In [17]:
ids_rule1.rule_overlap(ids_rule2, quant_dataframe_train)

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False,

In [18]:
type(data_train)

pandas.core.frame.DataFrame

In [19]:
type(mine_CARs(data_train, rule_cutoff=20)[0])

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=4
Rule count: 50, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 2
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 3
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 4
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 5
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000000000004, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=4
Rule count: 71, Iteration: 6
Decreasing confidence to 0.25000000000000006
Running apriori

pyarc.data_structures.car.ClassAssocationRule

In [20]:
ruleset = IDSRuleSet.from_cba_rules(cars)

In [21]:
ruleset.sum_rule_length(), ruleset.max_rule_length()

(46, 3)

In [22]:
ids_ruleset = IDSRuleSet.from_cba_rules(cars)

In [23]:
type(ids_ruleset)

pyids.data_structures.ids_ruleset.IDSRuleSet

In [24]:
params = ObjectiveFunctionParameters()

params.params["all_rules"] = ids_ruleset
params.params["len_all_rules"] = len(ids_ruleset.ruleset)
params.params["quant_dataframe"] = quant_dataframe_train
params.params["lambda_array"] = 7 * [1]


In [25]:
obj_func = IDSObjectiveFunction(params)

cover cache prepared
overlap cache prepared


In [26]:
rule_subset = IDSRuleSet(list(ids_ruleset.ruleset)[2:5])

In [27]:
rule_subset.ruleset

{IDSRule {Passenger_Cat=2nd_class,Gender=male,Age_Cat=adult} => {Died=1} sup: 0.08 conf: 0.91, f1: 0.15, len: 4, id: 43,
 IDSRule {Passenger_Cat=2nd_class,Age_Cat=adult} => {Died=1} sup: 0.09 conf: 0.82, f1: 0.16, len: 3, id: 42,
 IDSRule {Passenger_Cat=crew,Gender=male,Age_Cat=adult} => {Died=1} sup: 0.31 conf: 0.77, f1: 0.44, len: 4, id: 4}

In [28]:
obj_func.evaluate(ids_ruleset)

280486.0

In [29]:
rs_opt = RSOptimizer(input_set=ids_ruleset.ruleset)

solution_set = rs_opt.optimize()

In [30]:
solution_set

{IDSRule {Passenger_Cat=1st_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 4, id: 66,
 IDSRule {Gender=female,Passenger_Cat=crew} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 28,
 IDSRule {Passenger_Cat=2nd_class,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.02, len: 3, id: 69,
 IDSRule {Passenger_Cat=1st_class,Gender=female,Age_Cat=adult} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 4, id: 40,
 IDSRule {Passenger_Cat=2nd_class,Gender=male} => {Died=1} sup: 0.08 conf: 0.85, f1: 0.15, len: 3, id: 44,
 IDSRule {Gender=male,Age_Cat=adult,Passenger_Cat=3rd_class} => {Died=1} sup: 0.17 conf: 0.85, f1: 0.29, len: 4, id: 8,
 IDSRule {Passenger_Cat=3rd_class,Gender=male} => {Died=1} sup: 0.19 conf: 0.82, f1: 0.31, len: 3, id: 9,
 IDSRule {Passenger_Cat=3rd_class,Age_Cat=adult} => {Died=1} sup: 0.21 conf: 0.77, f1: 0.33, len: 3, id: 7,
 IDSRule {Passenger_Cat=crew,Gender=male,Age_Cat=adult} => {Died=1} sup: 0.31 conf: 0.77, f1: 0.44, le

In [31]:
obj_func.evaluate(IDSRuleSet(solution_set))

283950.0

In [32]:
dls_opt = DLSOptimizer(obj_func, params)

In [33]:
solution_set = dls_opt.optimize()

Testing if rule is good to add IDSRule {Passenger_Cat=2nd_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 68
Testing if rule is good to add IDSRule {Gender=female,Passenger_Cat=crew} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 28
Testing if rule is good to add IDSRule {Gender=male,Age_Cat=adult} => {Died=1} sup: 0.64 conf: 0.81, f1: 0.71, len: 3, id: 1
Testing if rule is good to add IDSRule {Passenger_Cat=3rd_class,Gender=male} => {Died=1} sup: 0.19 conf: 0.82, f1: 0.31, len: 3, id: 9
Testing if rule is good to add IDSRule {Passenger_Cat=2nd_class,Age_Cat=adult} => {Died=1} sup: 0.09 conf: 0.82, f1: 0.16, len: 3, id: 42
Testing if rule is good to add IDSRule {Passenger_Cat=crew,Gender=male,Age_Cat=adult} => {Died=1} sup: 0.31 conf: 0.77, f1: 0.44, len: 4, id: 4
Testing if rule is good to add IDSRule {Passenger_Cat=crew} => {Died=1} sup: 0.31 conf: 0.77, f1: 0.44, len: 2, id: 6
Testing if rule is good to add IDSRule {Passenger_Cat=1st

In [34]:
obj_func.evaluate(IDSRuleSet(solution_set))

288829.0

In [35]:
sls_opt = SLSOptimizer(obj_func, params)
solution_set = sls_opt.optimize()
obj_func.evaluate(IDSRuleSet(solution_set))

INFO - Number of input rules: 20
INFO - RandomOptimizer estimated the OPTIMUM value as: 286314.0
INFO - Threshold value (2/(n*n) * OPT) = 1431.57. This is the standard error treshold value.
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=2nd_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 68
INFO - stardard error of omega estimate: 366.47879201940185
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=3rd_class,Gender=male} => {Died=1} sup: 0.19 conf: 0.82, f1: 0.31, len: 3, id: 9
INFO - stardard error of omega estimate: 336.93827476260395
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=2nd_class,Age_Cat=adult} => {Died=1} sup: 0.09 conf: 0.82, f1: 0.16, len: 3, id: 42
INFO - stardard error of omega estimate: 268.5010055847091
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=crew,Gender=male,Age_Cat=adult} => {D

INFO - stardard error of omega estimate: 305.41533851462015
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=1st_class,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 67
INFO - stardard error of omega estimate: 264.8289447926718
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Gender=male,Age_Cat=adult,Passenger_Cat=3rd_class} => {Died=1} sup: 0.17 conf: 0.85, f1: 0.29, len: 4, id: 8
INFO - stardard error of omega estimate: 404.41178148021356
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Gender=female,Passenger_Cat=1st_class} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 3, id: 41
INFO - stardard error of omega estimate: 318.30306313323473
INFO - omega succesfully estimated


287226.0

In [36]:
ids_clf = IDSClassifier(solution_set)

ids_clf.quant_dataframe_train = quant_dataframe_train

ids_clf.calculate_default_class()

['0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1']


In [37]:
ids_clf.default_class

'1'

In [38]:
ids_clf.default_class_confidence

0.6436781609195402

In [39]:
ids_clf.score_auc(quant_dataframe)

AttributeError: 'IDSClassifier' object has no attribute 'score_auc'

In [40]:
import numpy as np

class RandomizedUSMOptimizer:

    def __init__(self, objective_function, objective_func_params, debug=True, random_seed=None):
        self.objective_function_params = objective_func_params 
        self.objective_function = objective_function
        self.debug = debug

        if random_seed:
            np.random.seed(random_seed)

    def optimize(self):
        all_rules = self.objective_function_params.params["all_rules"]

        x0 = IDSRuleSet(set())
        y0 = IDSRuleSet({rule for rule in all_rules.ruleset})

        n = len(y0)

        for rule in all_rules.ruleset:
            a_set = IDSRuleSet(x0.ruleset | {rule})
            b_set = IDSRuleSet(y0.ruleset - {rule})

            a_value = self.objective_function.evaluate(a_set) - self.objective_function.evaluate(x0)
            b_value = self.objective_function.evaluate(b_set) - self.objective_function.evaluate(y0)

            a_max = max(a_value, 0)
            b_max = max(b_value, 0)

            x_probability = 1
            y_probability = 0
            
            if not (a_max == 0 and b_max == 0):
                x_probability = a_max / (a_max + b_max)
                y_probability = b_max / (a_max + b_max)

            if np.random.uniform() <= x_probability:
                x0.ruleset.add(rule)

            if np.random.uniform() <= y_probability:
                y0.ruleset.remove(rule)                

        x_value = self.objective_function.evaluate(x0)
        y_value = self.objective_function.evaluate(y0)

        if x_value > y_value:
            return x0.ruleset
        else:
            return y0.ruleset

In [41]:
params.params["lambda_array"] = [0, 0, 1, 0, 0, 0, 0]

rusm_opt = RandomizedUSMOptimizer(obj_func, params)
solution_set = rusm_opt.optimize()
obj_func.evaluate(IDSRuleSet(solution_set))

140800.0

In [42]:
ids_ruleset.ruleset

{IDSRule {Passenger_Cat=1st_class,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 67,
 IDSRule {Passenger_Cat=1st_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 4, id: 66,
 IDSRule {Gender=female,Passenger_Cat=crew} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 28,
 IDSRule {Passenger_Cat=crew,Gender=female,Age_Cat=adult} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 4, id: 27,
 IDSRule {Gender=female,Passenger_Cat=2nd_class,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 70,
 IDSRule {Passenger_Cat=2nd_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 68,
 IDSRule {Passenger_Cat=2nd_class,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.02, len: 3, id: 69,
 IDSRule {Gender=female,Passenger_Cat=1st_class} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 3, id: 41,
 IDSRule {Passenger_Cat=1st_class,Gender=female,Age_Cat=adult} => {Died=0} sup: 0.06 c

In [43]:
solution_set

{IDSRule {Passenger_Cat=1st_class,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 67,
 IDSRule {Gender=female,Passenger_Cat=crew} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 28,
 IDSRule {Gender=female,Passenger_Cat=2nd_class,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 70,
 IDSRule {Gender=female,Passenger_Cat=1st_class} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 3, id: 41,
 IDSRule {Gender=male,Age_Cat=adult,Passenger_Cat=3rd_class} => {Died=1} sup: 0.17 conf: 0.85, f1: 0.29, len: 4, id: 8}

In [44]:
ids = IDS()
ids.fit(quant_dataframe_train, class_association_rules=cars)

cover cache prepared
overlap cache prepared
INFO - Number of input rules: 20
INFO - RandomOptimizer estimated the OPTIMUM value as: 286110.0
INFO - Threshold value (2/(n*n) * OPT) = 1430.55. This is the standard error treshold value.
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=2nd_class,Gender=male,Age_Cat=child} => {Died=0} sup: 0.01 conf: 1.00, f1: 0.01, len: 4, id: 68
INFO - stardard error of omega estimate: 325.01567654499377
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=3rd_class,Gender=male} => {Died=1} sup: 0.19 conf: 0.82, f1: 0.31, len: 3, id: 9
INFO - stardard error of omega estimate: 286.1392685389407
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=2nd_class,Age_Cat=adult} => {Died=1} sup: 0.09 conf: 0.82, f1: 0.16, len: 3, id: 42
INFO - stardard error of omega estimate: 377.8773716432356
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenge

INFO - stardard error of omega estimate: 284.7242964694091
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=1st_class,Gender=female,Age_Cat=adult} => {Died=0} sup: 0.06 conf: 1.00, f1: 0.11, len: 4, id: 40
INFO - stardard error of omega estimate: 265.28184257502437
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=3rd_class,Age_Cat=adult} => {Died=1} sup: 0.21 conf: 0.77, f1: 0.33, len: 3, id: 7
INFO - stardard error of omega estimate: 287.0047525738903
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Gender=male} => {Died=1} sup: 0.66 conf: 0.79, f1: 0.72, len: 2, id: 2
INFO - stardard error of omega estimate: 213.43023216030105
INFO - omega succesfully estimated
INFO - Estimating omega for rule: IDSRule {Passenger_Cat=1st_class,Age_Cat=child} => {Died=0} sup: 0.00 conf: 1.00, f1: 0.01, len: 3, id: 67
INFO - stardard error of omega estimate: 308.74894007915236
INFO - omega

In [45]:
ids.score(quant_dataframe_train)

0.7698863636363636

In [50]:
np.where([True, True, True, False, False, True], [1, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0])

array([1, 1, 1, 1, 1, 1])

In [49]:
ids.score_auc(quant_dataframe_train, confidence_based=True)

KeyError: 4

## Mining the Class Association Rules (CARs)

## Training the IDS Model

In [23]:
ids = IDS()
ids.fit(class_association_rules=rules, quant_dataframe=quant_dataframe_train, debug=True, random_seed=5)

NameError: name 'rules' is not defined

In [ ]:
print(print_cars(ids.clf.rules))

In [24]:
ids.clf.default_class

AttributeError: 'NoneType' object has no attribute 'default_class'

In [13]:
ids.clf.default_class_confidence

1

## Evaluating the IDS model

In [30]:
print("Accuracy on train data: ", ids.score(quant_dataframe_train))
print("Accuracy on test data: ", ids.score(quant_dataframe_test))

Accuracy on train data:  0.6590909090909091
Accuracy on test data:  0.681334279630944


In [15]:
print("AUC on train data: ", ids.score_auc(quant_dataframe_train, confidence_based=True))
print("AUC on test data: ", ids.score_auc(quant_dataframe_test, confidence_based=True))

AUC on train data:  0.6735958005249345


IndexError: index 352 is out of bounds for axis 1 with size 352

In [32]:
ids.score_interpretable_metrics(quant_dataframe_train)

{'fraction_overlap': 0.1952651515151515,
 'fraction_classes': 1.0,
 'fraction_uncovered': 0.14204545454545447,
 'average_rule_width': 1.8333333333333333,
 'ruleset_length': 6}

In [67]:
ids.score_interpretable_metrics(quant_dataframe_test)

{'fraction_overlap': 0.32008516678495386,
 'fraction_classes': 1.0,
 'fraction_uncovered': 0.00709723207948898,
 'average_rule_width': 1.4285714285714286,
 'ruleset_length': 7}